Instanciate image reader:

In [1]:
from signreader.detector import ImageReader

ireader = ImageReader(print=True)



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4 
platform:       	darwin 
python version: 	3.10.16 
torch version:  	2.7.1! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




Open image:

In [2]:
from PIL import Image

fname_img = '/Users/manu/boulot/unit_solutions/data/datasets/segmentation/cellpose/test/48159-1_2.jpg'
img_pil = Image.open(fname_img)

Run analysis:

In [3]:
icontent = ireader(img_pil)

Running sign detector...
Analyzing sign 1...
Running text reader...
Running symbol detector...

0: 1920x2560 1 bushof, 1846.4ms
Speed: 84.9ms preprocess, 1846.4ms inference, 7.2ms postprocess per image at shape (1, 3, 1920, 2560)
Putting all sign information together...
Analyzing sign 2...
Running text reader...
Running symbol detector...

0: 1920x2560 1 bushof, 1820.8ms
Speed: 85.0ms preprocess, 1820.8ms inference, 4.5ms postprocess per image at shape (1, 3, 1920, 2560)
Putting all sign information together...
Analyzing sign 3...
Running text reader...
Running symbol detector...

0: 1920x2560 1 bahnhof, 1 bushof, 1 strassenbahn, 1799.5ms
Speed: 92.3ms preprocess, 1799.5ms inference, 4.5ms postprocess per image at shape (1, 3, 1920, 2560)
Putting all sign information together...


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_detector_result(img_np, masks, icontent):
    mask_binary = masks >= 1

    # Multiply image by mask, so we see only signs. Everything else is blacked out
    img_np_focus = np.zeros(img_np.shape, dtype=np.uint8)
    for channel in range(3):
        img_np_focus[:, :, channel] = img_np[:, :, channel] * mask_binary

    fig, axes = plt.subplots(1,3)
    axes[0].imshow(img_np_focus)
    axes[0].set_title('Input image')

    axes[1].imshow(img_np_focus)
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        confidence = float(box.conf[0])
        class_id = int(box.cls[0])
        label = class_names[class_id]

        # Ajouter une boîte rectangle
        width, height = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), width, height,
                                 linewidth=1, edgecolor='magenta', facecolor='none')
        ax.add_patch(rect)

        # Ajouter le texte (label + score)
        text = f"{label} {confidence:.2f}"
        ax.text(x1, y1 - 5, text,
            color='magenta',
            fontsize='xx-small')